**Get segmentations instead of boxes ?**

# Install detectron2

**References :** https://github.com/facebookresearch/detectron2


In [ ]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

# Properly install detectron2. (Please do not install twice in both ways)
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [2]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
torch:  2.1 ; cuda:  cu118
detectron2: 0.6


**Libs**

In [3]:
# Some basic setup:
# Setup detectron2 logger
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.utils.visualizer import ColorMode
import os


##**Personal Goal ❎**

**Here I Am going to personalize the work and get only Human detection as output.**

**The input image will be cropped into N persons detected by Detectron2**

**We will be adding variable N to calculate the occurence of the Humans in the pic**

# Run a pre-trained detectron2 model

Then, we create a detectron2 config and a detectron2 `DefaultPredictor` to run inference on this image.

In [4]:
cfg = get_cfg()

# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model

# Load Model
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")

# Create a predictor
predictor = DefaultPredictor(cfg)

[11/15 20:09:51 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl ...


model_final_f10217.pkl: 178MB [00:01, 139MB/s]                           


-> so 0 stands for person

**FILTER PERSON ONLY**

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
drive_image_dir ="/content/drive/MyDrive/imgs_test_set" #input Images
output_directory="/content/drive/MyDrive/imgs_test_set/Detectron2_result/segmented_humans" #output Folder
#make sure we have output dir
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

**Multiply the mask with the original image to extract the segmented person**

In [11]:
# List all image files in the directory in Google Drive
image_files = [os.path.join(drive_image_dir, filename) for filename in os.listdir(drive_image_dir) if filename.endswith(".jpeg")]

for image_path in image_files:
    # Load an image from Google Drive
    im = cv2.imread(image_path)

    # Perform inference on the loaded image
    outputs = predictor(im)

    # Initialize the number of humans detected in the current image
    n_human = 0

    # Create a copy of the original image
    image_with_results = im.copy()

    # Filter predictions to only include "person" class
    filtered_instances = outputs["instances"][outputs["instances"].pred_classes == MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).thing_classes.index("person")]

    # Update the number of humans detected
    n_human = len(filtered_instances)

    # Save segmentation masks of each detected person
    for i, mask in enumerate(filtered_instances.pred_masks):
        # Convert the PyTorch tensor to a NumPy array
        mask_np = mask.cpu().numpy().astype(np.uint8)

        # Multiply the mask with the original image to extract the segmented person
        segmented_person = cv2.bitwise_and(im, im, mask=mask_np)

        # Get the original image name without the extension
        image_name = os.path.splitext(os.path.basename(image_path))[0]

        # Save the segmented person
        save_path = os.path.join(output_directory, f"{image_name}_person_{i}_segmentation.jpg")
        cv2.imwrite(save_path, segmented_person)

    # Print the number of humans detected in the current image
    print(f"Number of humans in {image_path}: {n_human}")


Number of humans in /content/drive/MyDrive/imgs_test_set/1.jpeg: 11
Number of humans in /content/drive/MyDrive/imgs_test_set/2.jpeg: 5
Number of humans in /content/drive/MyDrive/imgs_test_set/3.jpeg: 8
Number of humans in /content/drive/MyDrive/imgs_test_set/4.jpeg: 2
Number of humans in /content/drive/MyDrive/imgs_test_set/5.jpeg: 13
Number of humans in /content/drive/MyDrive/imgs_test_set/boy-1822565_1280.jpeg: 1
Number of humans in /content/drive/MyDrive/imgs_test_set/cmxseT20wDlQ_s600x600.jpeg: 6
Number of humans in /content/drive/MyDrive/imgs_test_set/coachplayertf.jpeg: 2
Number of humans in /content/drive/MyDrive/imgs_test_set/fef23683-0b70-49d7-8b32-d11ae842f633.jpeg: 3
Number of humans in /content/drive/MyDrive/imgs_test_set/images (1).jpeg: 9
Number of humans in /content/drive/MyDrive/imgs_test_set/input - Copie.jpeg: 5


**BLACK AND WHITE ?**

In [10]:
# List all image files in the directory in Google Drive
image_files = [os.path.join(drive_image_dir, filename) for filename in os.listdir(drive_image_dir) if filename.endswith(".jpeg")]

for image_path in image_files:
    # Load an image from Google Drive
    im = cv2.imread(image_path)

    # Perform inference on the loaded image
    outputs = predictor(im)

    # Initialize the number of humans detected in the current image
    n_human = 0

    # Create a copy of the original image
    image_with_results = im.copy()

    # Filter predictions to only include "person" class
    filtered_instances = outputs["instances"][outputs["instances"].pred_classes == MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).thing_classes.index("person")]

    # Update the number of humans detected
    n_human = len(filtered_instances)

    # Save segmentation masks of each detected person
    for i, mask in enumerate(filtered_instances.pred_masks):
        # Convert the PyTorch tensor to a NumPy array and then to a 3-channel mask
        mask = np.repeat(mask.cpu().numpy()[:, :, np.newaxis], 3, axis=2) * 255

        # Get the original image name without the extension
        image_name = os.path.splitext(os.path.basename(image_path))[0]

        # Save the segmentation mask in your Google Drive directory with the original image name and person index
        save_path = os.path.join(output_directory, f"{image_name}_person_{i}_mask.jpg")
        cv2.imwrite(save_path, mask.astype(np.uint8))

    # Print the number of humans detected in the current image
    print(f"Number of humans in {image_path}: {n_human}")

Number of humans in /content/drive/MyDrive/imgs_test_set/1.jpeg: 11
Number of humans in /content/drive/MyDrive/imgs_test_set/2.jpeg: 5
Number of humans in /content/drive/MyDrive/imgs_test_set/3.jpeg: 8
Number of humans in /content/drive/MyDrive/imgs_test_set/4.jpeg: 2
Number of humans in /content/drive/MyDrive/imgs_test_set/5.jpeg: 13
Number of humans in /content/drive/MyDrive/imgs_test_set/boy-1822565_1280.jpeg: 1
Number of humans in /content/drive/MyDrive/imgs_test_set/cmxseT20wDlQ_s600x600.jpeg: 6
Number of humans in /content/drive/MyDrive/imgs_test_set/coachplayertf.jpeg: 2
Number of humans in /content/drive/MyDrive/imgs_test_set/fef23683-0b70-49d7-8b32-d11ae842f633.jpeg: 3
Number of humans in /content/drive/MyDrive/imgs_test_set/images (1).jpeg: 9
Number of humans in /content/drive/MyDrive/imgs_test_set/input - Copie.jpeg: 5
